# Trabajo práctico Nº 5 

## Ejercicio 2 

Se implementa el mismo sistema de comunicación simplificado que en el ejercicio anterior aunque de una forma más ligada a la implementación en hardware. Para así poder generar los archivos de vectores, probar en varios puntos la señal, etc. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tool.ff_model  as fm
from tool._fixedInt import * 
from tool.DSPtools import *

Definimos los mismos parámetros que para el ejercicio anterior, agregando algunos extras, como el **orden de PSBR** a implementar emulando una LSFR 

In [2]:
clk = 100e6                 #Clock Freq.  
N = 4                       #Oversampling 
BR = clk / N                #Baud Rate 
rolloff = 0.5               
fs = N*BR                   #Sample Freq. 
Ts = 1/fs                   #Sample T. 
T = 1/BR                    #Symbol sep 
M = 2                       #Symbols (levels) 
Nsymb = 1e3                 #Simulation Length 
NB = 6                      #RC taps
PRBSn = 9                   #PRBS order

### 1) Generamos Pseudo Random Bit Sequence 

Modelo de implementación con LFSR 
![LFSR](./imgs/lfsr.png)

**NOTA**: Se agrega una pequeña modificación al módulo ff para que pueda recibir enteros en cualquier base (decimal, hexa, bin) y cargue la representación en bits a cada uno de los ff 

```
binList = [1 if digit=='1' else 0 for digit in bin(lfsrISeed)[2:]] 
```

In [17]:
lfsrI = fm.ff(PRBSn)
lfsrQ = fm.ff(PRBSn)
#lfsrISeed=np.array([1,1,0,1,0,1,0,1,0])
#lfsrQSeed=np.array([1,1,1,1,1,1,1,1,0])
lfsrISeed=0x1AA            #El modulo ff tambien recibe
lfsrQSeed=0x1FE            #enteros ahora, y los desglosa en bits

#Secuencia creada a partir de la LFSR 
PRBSeqI= []
PRBSeqQ= []

In [18]:
#LFSR siguiendo el modelo de ejemplo 
lfsrI.i = lfsrISeed
lfsrQ.i = lfsrQSeed
lfsrI.run_clock()
lfsrQ.run_clock()

for clkPulse in range(int(Nsymb)):
    lfsrI[0].i =  lfsrI[4].o ^ lfsrI[8].o       #Suma - XOR 
    lfsrQ[0].i =  lfsrQ[4].o ^ lfsrQ[8].o       
    
    for p in range(1,PRBSn):                    #Desplazamiento
        lfsrI[p].i = lfsrI[p-1].o
        lfsrQ[p].i = lfsrQ[p-1].o
    
    lfsrI.run_clock()
    lfsrQ.run_clock()
    
    PRBSeqI.append(lfsrI[PRBSn-1].o)
    PRBSeqQ.append(lfsrQ[PRBSn-1].o)

In [20]:
print(PRBSeqI[:20])
print(PRBSeqQ[:20])

[1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1]


[1, 1, 0, 1, 0, 1, 0, 1, 0]
